In [4]:
import sys
from math import log

In [41]:
# make test
with open('test1.txt', 'w') as test:
    test.write('500000\n')
    for i in range(500000):
        test.write(str(np.random.randint(100000)) + '\n')

In [67]:
# naive (memory limit)
unique_queries = set()

test = open('test1.txt') # sys.stdin
n = int(test.readline().strip())
for line in test:
    unique_queries.add(line.strip())
test.close()

n, len(unique_queries)

peak memory: 110.46 MiB, increment: 3.50 MiB


In [75]:
# HyperLogLog

class HLL(object):
    """ 
    HyperLogLog
    source: https://github.com/clarkduvall/hypy
    paper:  http://algo.inria.fr/flajolet/Publications/FlFuGaMe07.pdf
    """
    P32 = 2 ** 32

    def __init__(self, p=14):
        self.p, self.m, self.r = p, 1 << p, [0] * (1 << p)

    def add(self, x):
        x = hash(x)
        i = x & HLL.P32 - 1 >> 32 - self.p
        z = 35 - len(bin(HLL.P32 - 1 & x << self.p | 1 << self.p - 1))
        self.r[i] = max(self.r[i], z)

    def count(self):
        a = ({16: 0.673, 32: 0.697, 64: 0.709}[self.m]
             if self.m <= 64 else 0.7213 / (1 + 1.079 / self.m))
        e = a * self.m * self.m / sum(1.0 / (1 << x) for x in self.r)
        if e <= self.m * 2.5:
            z = len([r for r in self.r if not r])
            return int(self.m * log(float(self.m) / z) if z else e)
        return int(e if e < HLL.P32 / 30 else -HLL.P32 * log(1 - e / HLL.P32))

def read_stdin():
    """ stdin generator reader """
    readline = sys.stdin.readline()
    while readline:
        yield readline
        readline = sys.stdin.readline()
        
def read_stdin(file):
    """ file generator reader (for test) """
    readline = file.readline().strip()
    while readline:
        yield readline
        readline = file.readline().strip()        

h = HLL()

test = open('test1.txt') # sys.stdin
n = int(test.readline().strip())
for line in read_stdin(test):
    # Add twice to make sure it only gets counted once.
    h.add(hash(line))
    h.add(hash(line))
test.close()

print(h.count())

97820
